# ERA 5

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import time

path = 'mypath'

In [2]:
df_metadata_train = pd.read_csv(path + 'df_metadata_train.csv', index_col=0)
df_metadata_train.index = df_metadata_train.index.astype(str)

df_metadata_test = pd.read_csv(path + 'df_metadata_test.csv', index_col=0)
df_metadata_test.index = df_metadata_test.index.astype(str)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [5]:
xr_train=xr.open_dataset(path+'train_sin_cos.netcdf')
xr_test=xr.open_dataset(path+'test_sin_cos.netcdf')

In [6]:
df_distances_test = pd.read_csv(path+'df_distances_test.csv', index_col=0)
df_distances_train = pd.read_csv(path+'df_distances_train.csv', index_col=0)
df_distances_test.index = df_distances_test.index.astype(str)
df_distances_train.index = df_distances_train.index.astype(str)

In [7]:
from torch.utils.data import Dataset


def get_seq_length(past_dict = {'year': 0, 'month': 0, 'week': 0, 'day': 1, 'hour': 0},
                   future_dict = {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': 1},
                   max_len = 133992):
  look_up = (past_dict['hour'] * 4 + past_dict['day'] * 24 * 4
            + past_dict['month'] *30 * 24 * 4 + past_dict['year'] * 12 * 30 * 24 * 4)
  look_ahead = (future_dict['hour'] * 4 + future_dict['day'] * 24 * 4
                + future_dict['month'] *30 * 24 * 4 + future_dict['year'] * 12 * 30 * 24 * 4)

  if look_up+look_ahead > max_len:
    print('Not enough data to accomodate requested sequence lengths, returning 1 and 1')
    look_up, look_ahead = 1, 1

  return look_up, look_ahead


class PVDataset_ERA(Dataset):
  def __init__(self, X,
               era, df_metadata, df_distances=None, look_up=1, look_ahead=1, with_neighbours=False,
               cut_off_km=64, n_neighbours=8):

    '''
    X = xarray dataset
    look_up = int, length of sequence of past observations to use for prediction
    look_ahead = int, length of sequence to predict
    era = xarray dataset of era5 data
    '''

    self.X = X
    self.stations = list(X.keys())
    self.look_up = look_up
    self.look_ahead = look_ahead
    self.shape_0 = X.dims['datetime']
    self.shape_1 = len(self.stations)
    self.data_len = self.shape_0 // (self.look_up + self.look_ahead)
    self.df_metadata = df_metadata
    self.era_keys = list(era.keys())
    self.era = era

    self.with_neighbours = with_neighbours
    if with_neighbours:
      self.df_distances = df_distances
      self.cut_off_km = cut_off_km
      self.n_neighbours = n_neighbours

  def __len__(self):
    return self.data_len * self.shape_1

  def __getitem__(self, i):

    def get_neighbours(ss_id):
      distances = self.df_distances[ss_id].sort_values().copy()
      distances = distances[distances > self.cut_off_km].copy()

      if len(distances) < self.n_neighbours:
        return np.random.choice(distances.index.values, self.n_neighbours, replace=True)

      else:
        return distances.index.values[list(range(0, len(distances), len(distances)//self.n_neighbours))[:self.n_neighbours]]


    ss_num = i // self.data_len
    ss_id = self.stations[ss_num]
    i = (i % self.shape_1) * (self.look_up + self.look_ahead)

    t = i + self.look_up

    while sum(self.X[ss_id].data[t-self.look_up : t]) < 0.5:
      i = np.random.choice(range(self.__len__()), 1)[0]

      ss_num = i // self.data_len
      ss_id = self.stations[ss_num]
      i = (i % self.shape_1) * (self.look_up + self.look_ahead)

      t = i + self.look_up

    x_item = np.zeros((self.look_up, 5+len(self.era_keys)))
    y_item = np.zeros((self.look_ahead, 1))

    if self.with_neighbours:
      x_item = np.zeros((self.look_up, 5+len(self.era_keys)+self.n_neighbours))
      neighbours = get_neighbours(ss_id)

    x_item[:, 0], y_item = self.X[ss_id].data[t-self.look_up : t], self.X[ss_id].data[t : t+self.look_ahead]

    era_slice = self.era.sel(datetime=self.X['datetime'].data[t-self.look_up : t], ss_id=ss_id).copy()

    for i, key in enumerate(self.era_keys):
      x_item[:, i+1] = era_slice[key].data

    if self.with_neighbours:
      for i, ss in enumerate(neighbours):
        x_item[:, i+1+len(self.era_keys)] = self.X[ss].data[t-self.look_up : t]

    x_item[:, -4] = self.X.date_sin.data[t-self.look_up : t]
    x_item[:, -3] = self.X.date_cos.data[t-self.look_up : t]
    x_item[:, -2] = self.X.time_sin.data[t-self.look_up : t]
    x_item[:, -1] = self.X.time_sin.data[t-self.look_up : t]

    return torch.Tensor(x_item).float(), torch.Tensor(y_item).float()

In [8]:
from torch.utils.data import DataLoader

def loaders(xr_test, xr_train, batch_size, dict_look_up, dict_look_ahead,
            df_metadata_train, df_metadata_test,
            cut_off_km=64, n_neighbours=8,
            df_distances_train=None, df_distances_test=None,
            demo=False, with_neighbours=False, era=None):

  if with_neighbours:
    if df_distances_train is None:
      df_distances_train = df_of_neighbours(df_metadata_train, df_metadata_train).copy()
    if df_distances_test is None:
      df_distances_test = df_of_neighbours(df_metadata_test, df_metadata_train).copy()

  max_len = len(xr_train['datetime'].data)

  look_up, look_ahead = get_seq_length(past_dict=dict_look_up, future_dict=dict_look_ahead, max_len=max_len)

  train_dataset = PVDataset_ERA(X=xr_train, df_distances=df_distances_train, look_up=look_up, look_ahead=look_ahead,
                            cut_off_km=cut_off_km, n_neighbours=n_neighbours,
                            df_metadata=df_metadata_train, era=era, with_neighbours=with_neighbours)
  test_dataset = PVDataset_ERA(X=xr_test, df_distances=df_distances_test, look_up=look_up, look_ahead=look_ahead,
                            cut_off_km=cut_off_km, n_neighbours=n_neighbours,
                            df_metadata=df_metadata_test, era=era, with_neighbours=with_neighbours)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  print("Expected number of steps is %d" %((train_dataset.__len__()  + batch_size - 1) // batch_size))

  return train_loader, test_loader, look_ahead

In [9]:
class LSTM_model(nn.Module):

  def __init__(self, hidden_size,
               num_layers,
               out_features,
               input_size,
               dropout=0):

    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        dropout=dropout
                        )

    self.fc = nn.Linear(in_features=hidden_size, out_features=out_features)

  def forward(self, x):

    batch_size = x.size(0)

    h_t = torch.zeros(self.num_layers,
                      batch_size,
                      self.hidden_size).to(device)
    c_t = torch.zeros(self.num_layers,
                      batch_size,
                      self.hidden_size).to(device)


    _, (h_out, _) = self.lstm(x, (h_t, c_t))

    h_out = h_out[0].view(-1, self.hidden_size)

    out = self.fc(h_out)

    return out

In [10]:
def train_epoch(model, train_loader, optimizer, loss_function, print_every=500):
  loss_array = []
  model.train(True)
  running_loss = 0.

  for batch_index, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    output = model(x_batch)

    loss = loss_function(output, y_batch.squeeze())
    running_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_index % print_every == 0:
      loss_array.append(running_loss/print_every)
      print("Batch %d, loss: %f" %(batch_index+1, running_loss/print_every))
      running_loss=0.

  loss_array.append(running_loss/((batch_index+1)%1000))

  print()
  return loss_array

In [11]:
def validate_epoch(model, test_loader, loss_function):
  model.train(False)
  running_loss = 0.

  for batch_index, batch in enumerate(test_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    with torch.no_grad():
      output = model(x_batch)
      loss = loss_function(output, y_batch.squeeze())
      running_loss += loss.item()

  avg_loss = running_loss / len(test_loader)

  print('Val loss: {0:.5f}'.format(avg_loss))
  print('_______________________________')
  print()

  return avg_loss

In [13]:
best_params_df = pd.read_csv(path+'era_neighbours_best_params.csv')
best_params = best_params_df.loc[0].to_dict()

In [14]:
best_params

{'num_layers': 2.0,
 'hidden_size': 256.0,
 'look_up': 18.0,
 'dropout': 0.9905943443390038,
 'cut_off_km': 128.0,
 'n_neighbours': 7.0}

In [24]:
n_neighbours = int(best_params['n_neighbours'])

data_params = {
    'xr_test': xr_test_sample,
    'xr_train': xr_train_sample,
    'batch_size': 128,
    'dict_look_up': {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': int(best_params['look_up'])},
    'dict_look_ahead': {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': 6},
    'cut_off_km': int(best_params['cut_off_km']),
    'n_neighbours': n_neighbours,
    'df_metadata_train': df_metadata_train,
    'df_metadata_test': df_metadata_test,
    'df_distances_train': df_distances_train_sample,
    'df_distances_test':df_distances_test_sample,
    'with_neighbours': True,
    'era': era
    }

train_loader, test_loader, look_ahead = loaders(**data_params)

Expected number of steps is 2289


In [25]:
model_params = {
    'hidden_size': int(best_params['hidden_size']),
    'num_layers': int(best_params['num_layers']),
    'out_features': look_ahead,
    'input_size': n_neighbours+1+4+3,
    'dropout': int(best_params['dropout'])
}

model = LSTM_model(**model_params)

model.to(device)

LSTM_model(
  (lstm): LSTM(15, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=24, bias=True)
)

In [ ]:
learning_rate = 0.001
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 10

train_loss = []
test_loss = []

for epoch in range(num_epochs):
  print("Epoch %d:" %(epoch+1))
  train_loss.append(train_epoch(model, train_loader, optimizer, loss_function, print_every=100))
  test_loss.append(validate_epoch(model, test_loader, loss_function))

torch.save(model.state_dict(), path+'ERA_neighbours_model')